# This script annotates transcripts of video simulation role-playing exercises from job candidates using large language models

In [28]:
# Install libraries that might not be installed as necessary (adjust manually)
# !pip install openai
# !pip install pandas
# !pip install pyyaml

In [29]:
# Import necessary libraries
import openai          # OpenAI API for AI model access
from openai import OpenAI  # The main client class for interacting with OpenAI's API
import requests        # HTTP library for making web requests (API calls, downloading data, etc.)
import re              # Regular expressions
import os              # Operating system interface
import pandas as pd    # DataFrames for tabular data manipulation and analysis
import numpy as np     # Numerical computing
import time            # Time-related functions
import random          # Random number generation
import yaml

# set seed for reproducibility
random.seed(1337)  # Sets seed for Python's random module

In [30]:
# Load data
# Advisement 2
advise = pd.read_csv("data/input/data_advise_text.csv")
# advise = advise.iloc[:2] # keep only first two rows for pilot data analysis purposes
advise = advise.iloc[[0, 16]]
advise.head()

# # Change management 2
# change_manage = pd.read_csv("data_change_manage_text.csv")
# # Team management 2
# team_manage = pd.read_csv("data_team_manage_text.csv")
# # Team work 2
# team_work = pd.read_csv("data_team_work_text.csv")

,id,Beoordeling.607,Beoordeling.608,Beoordeling.609,Beoordeling.610,Beoordeling.all
0,1460133,"Oh ja, Carina ik kan me voorstellen dat je voo...","Jeetje ik, ik kan me voorstellen dat je dat la...","Ja, dus als ik het zo hoor, dan is er dus eige...","Ik denk dat het goed is als we, zoals ik net a...","Oh ja, Carina ik kan me voorstellen dat je voo..."
16,1595547,"Dacht, Surina ja, ik begrijp je heel erg goed ...","Ja, ik begrijp je heel erg goed dat het best w...","Ja, ik zie ook echt wel dat het wat met je doe...",Maar wat ik zou aanraden is in ieder geval alv...,"Dacht, Surina ja, ik begrijp je heel erg goed ..."


In [31]:
# Call api key from config.yalm file
api_key = yaml.safe_load(open('config.yaml')) ['api_key']
# Import OpenAI API key
client = OpenAI(api_key=api_key)

# specify model used (e.g., gpt-4o, o3)
model_used = "gpt-4o"

In [32]:
# # See available openai models
# models = client.models.list()

# # Print model IDs (no truncation)
# for model in models.data:
#     print(model.id)

## Function to get ratings from openai API

In [33]:
# Manually change `model` to another openai model (as desired)
def get_rating(client, text, prompt_template, model):
    """
    Query the OpenAI model for a 1–5 rating based on a formatted prompt with transcript text.
    
    Parameters:
    - client: OpenAI client instance
    - text (str): The transcript to evaluate
    - prompt_template (str): A prompt string with {text} placeholder for transcript
    - model (str): Model name 

    Returns:
    - int or None: Extracted rating or None if parsing fails
    """
    try:
        full_prompt = prompt_template.format(text=text.strip())

        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "Je bent een beoordelaar die competenties beoordeelt op basis van gespreksfragmenten."},
                {"role": "user", "content": full_prompt}  # no additional input
            ],
            temperature=0.7, # lower values reduce randomness (0.7 matches the ChatGPT behavior)
            max_tokens=6  # we expect only a single digit
        )

        rating_text = response.choices[0].message.content.strip()
        match = re.search(r'\b([1-5](?:\.\d)?)\b', rating_text)
        if match:
            return int(match.group(0))
        else:
            print(f"Warning: Non-numeric or out-of-range response: '{rating_text}'")
            return None

    except Exception as e:
        print(f"Error processing text: {e}")
        return None

# Prompts

In [34]:
# Define general instructions for competence annotation
context = """
**Context van de rollenspel-oefening:**
Je bent een professionele recruiter. Jouw taak is om een sollicitant te beoordelen op basis van hun reacties 
in een rollenspel-oefening. In deze oefening reageert de kandidaat verbaal op een fictieve collega genaamd Lara, in vier scènes. 
Lara deelt professionele frustraties, fouten in klantprojecten, emotionele uitputting 
en persoonlijke financiële zorgen, en ze vraagt de kandidaat om steun en advies. Samenvattend:

- **Scène 1**: Lara uit haar frustratie over dat ze op het werk wordt overgeslagen, repetitieve taken moet doen, 
en geen groeimogelijkheden heeft. Ze vraagt om advies.
- **Scène 2**: Lara bekent dat ze twee klantprojecten verkeerd heeft aangepakt, wat kostenproblemen en ontevreden klanten heeft veroorzaakt. 
Ze is bang om dit aan de teamleider te vertellen en vraagt of de kandidaat namens haar kan spreken.
- **Scène 3**: Lara onthult emotionele uitputting en financiële stress door de chronische ziekte van haar dochter. 
Ze vraagt zich af of ze extra verlof kan krijgen of dat de kandidaat enkele taken kan overnemen.
- **Scène 4**: Lara voelt zich iets beter, maar is nog steeds overweldigd. Ze vraagt de kandidaat om concreet advies over wat ze nu moet doen.
"""

instructions = """
**Reactie van de kandidaat:**
Je krijgt hieronder het transcript van de volledige reactie van de kandidaat in alle vier scènes.

**Beoordelingsinstructies:**
1. Lees het transcript van de reactie van de kandidaat (afgebakend tussen de drie backticks).
2. Let op **verbale gedragingen, waarden of attitudes** die deze competentie weerspiegelen.
3. Beoordeel de kandidaat op een **schaal van 1 tot 5**, met behulp van de volgende rubric:

- **1 = Onvoldoende**: De kandidaat toont geen of nauwelijks vaardigheid in deze competentie. De reactie is ongepast, onsamenhangend of afwezig.
- **2 = Matig**: De kandidaat laat beperkte vaardigheid zien. De poging is oppervlakkig, niet effectief of mist richting.
- **3 = Voldoende**: De kandidaat toont een basaal niveau van de competentie. De reactie is relevant, maar mist diepgang of consistentie.
- **4 = Goed**: De kandidaat past de competentie duidelijk en adequaat toe. De reactie is doordacht, gestructureerd en passend bij de situatie.
- **5 = Uitstekend**: De kandidaat blinkt uit in het toepassen van de competentie. De reactie is scherp, empathisch, effectief en getuigt van inzicht en initiatief.

Welk cijfer past het best bij deze reactie? Kies: 1, 2, 3, 4 of 5.
Gebruik de volledige schaal van 1 tot 5. Beoordeel niet automatisch gemiddeld. 
**Geef alleen een enkel cijfer van 1 tot 5 als output. Geef geen uitleg.**

```{text}```
"""

# Prompts for rating job competencies
promptAD_PROBL_I1 = f"""
{context}
**Te beoordelen competentie:**  
Beoordeel **in welke mate de kandidaat onderscheid maakt tussen hoofd- en bijzaken** in hun reactie op het verhaal van Lara.
{instructions}
"""

promptAD_PROBL_I2 = f"""
{context}
**Te beoordelen competentie:**  
Beoordeel **in welke mate de kandidaat verbanden legt tussen verschillende aspecten van de vraagstelling** in hun reactie op het verhaal van Lara.
{instructions}
"""

promptAD_PROBL_I3 = f"""
{context}
**Te beoordelen competentie:**  
Beoordeel **in welke mate de kandidaat de kern van de informatie benoemt** in hun reactie op het verhaal van Lara.
{instructions}
"""

promptAD_CREAT_I1 = f"""
{context}
**Te beoordelen competentie:**  
Beoordeel **in welke mate de kandidaat met originele ideeën komt als reactie** in hun reactie op het verhaal van Lara.
{instructions}
"""

promptAD_CREAT_I2 = f"""
{context}
**Te beoordelen competentie:**  
Beoordeel **in welke mate de kandidaat zaken benadert vanuit meerdere invalshoeken bij het bedenken van oplossingen of ideeën** in hun reactie op het verhaal van Lara.
{instructions}
"""

promptAD_CREAT_I3 = f"""
{context}
**Te beoordelen competentie:**  
Beoordeel **in welke mate de kandidaat bestaande zaken combineert tot iets nieuws in de reactie** in hun reactie op het verhaal van Lara.
{instructions}
"""

promptAD_OORDE_I1 = f"""
{context}
**Te beoordelen competentie:**  
Beoordeel **in welke mate de kandidaat in zijn/haar/hen beeldvorming een onderscheid maakt tussen relevante en niet-relevante aspecten** in hun reactie op het verhaal van Lara.
{instructions}
"""

promptAD_OORDE_I2 = f"""
{context}
**Te beoordelen competentie:**  
Beoordeel **in welke mate de kandidaat zijn/haar/hen redenering met feiten onderbouwt in de reactie** in hun reactie op het verhaal van Lara.
{instructions}
"""

promptAD_OORDE_I3 = f"""
{context}
**Te beoordelen competentie:**  
Beoordeel **in welke mate de kandidaat van zijn/haar/hen eigen standpunt zowel de voor- als nadelen benoemt** in hun reactie op het verhaal van Lara.
{instructions}
"""

promptAD_ORGANS_I1 = f"""
{context}
**Te beoordelen competentie:**  
Beoordeel **in welke mate de kandidaat in het gesprek rekening houdt met gevoeligheden van anderen in de organisatie**, in hun reactie op het verhaal van Lara.
{instructions}
"""

promptAD_ORGANS_I2 = f"""
{context}
**Te beoordelen competentie:**  
Beoordeel **in welke mate de kandidaat vooraf toetst hoe acties of besluiten binnen de organisatie ontvangen zullen worden**, in hun reactie op het verhaal van Lara.
{instructions}
"""

promptAD_ORGANS_I3 = f"""
{context}
**Te beoordelen competentie:**  
Beoordeel **in welke mate de kandidaat de gevolgen voor andere personen of organisatie-onderdelen onderzoekt alvorens een besluit te nemen**, in hun reactie op het verhaal van Lara.
{instructions}
"""


In [35]:
# Initialize new columns to store the annotations

# Specify which LLM model is being used for annotations
# This will be prefixed to all generated column names (e.g., "4o_rating607A")

# Dictionary mapping source columns to their respective rating columns and prompts
annotation_config = {
    "Beoordeling.all": [
        (f"{model_used}_AD_PROBL_I1", promptAD_PROBL_I1),
        (f"{model_used}_AD_PROBL_I2", promptAD_PROBL_I2),
        (f"{model_used}_AD_PROBL_I3", promptAD_PROBL_I3),
        (f"{model_used}_AD_CREAT_I1", promptAD_CREAT_I1),
        (f"{model_used}_AD_CREAT_I2", promptAD_CREAT_I2),
        (f"{model_used}_AD_CREAT_I3", promptAD_CREAT_I3),
        (f"{model_used}_AD_OORDE_I1", promptAD_OORDE_I1),
        (f"{model_used}_AD_OORDE_I2", promptAD_OORDE_I2),
        (f"{model_used}_AD_OORDE_I3", promptAD_OORDE_I3),
        (f"{model_used}_AD_ORGANS_I1", promptAD_ORGANS_I1),
        (f"{model_used}_AD_ORGANS_I2", promptAD_ORGANS_I2),
        (f"{model_used}_AD_ORGANS_I3", promptAD_ORGANS_I3)
    ]
}

# Initialize all rating columns with NaN values
# Creates columns in format: "{model_used}_ratingXXX"
for ratings in annotation_config.values():
    for rating_col, _ in ratings:
        advise[rating_col] = np.nan

# Verify new columns were created successfully
# display(advise.head())


# Text annotation

In [36]:
# Process each row in the advise DataFrame
for idx, row in advise.iterrows():
    try:
        # Loop through each Beoordeling column and its associated ratings/prompts
        for beoordeling_col, ratings in annotation_config.items():
            
            # Extract the transcript
            text = row.get(beoordeling_col, "")
            
            if pd.notna(text) and text.strip() != "":
                for rating_col, prompt_template in ratings:
                    
                    # Query the model with formatted prompt
                    rating = get_rating(client, text, prompt_template, model = model_used)
                    
                    # Save the rating
                    advise.loc[idx, rating_col] = rating
                    
                    # Add small delay to respect rate limits
                    time.sleep(0.5)

    except Exception as e:
        print(f"Error processing row {idx}: {e}")
        continue

    # Progress logging
    if (idx + 1) % 5 == 0:
        print(f"Processed {idx + 1}/{len(advise)} rows")


In [37]:
# Calculate the average for each of the four sub-competencies and store in new columns
advise[f"{model_used}_AD_PROBL"] = advise[[f"{model_used}_AD_PROBL_I1", f"{model_used}_AD_PROBL_I2", f"{model_used}_AD_PROBL_I3"]].mean(axis=1)
advise[f"{model_used}_AD_CREAT"] = advise[[f"{model_used}_AD_CREAT_I1", f"{model_used}_AD_CREAT_I2", f"{model_used}_AD_CREAT_I3"]].mean(axis=1)
advise[f"{model_used}_AD_OORDE"] = advise[[f"{model_used}_AD_OORDE_I1", f"{model_used}_AD_OORDE_I2", f"{model_used}_AD_OORDE_I3"]].mean(axis=1)
advise[f"{model_used}_AD_ORGANS"] = advise[[f"{model_used}_AD_ORGANS_I1", f"{model_used}_AD_ORGANS_I2", f"{model_used}_AD_ORGANS_I3"]].mean(axis=1)

# Calculate the average of the 4 new columns
advise[f"{model_used}_AD"] = advise[[f"{model_used}_AD_PROBL", f"{model_used}_AD_CREAT", f"{model_used}_AD_OORDE", f"{model_used}_AD_ORGANS"]].mean(axis=1)

advise.head()

,id,Beoordeling.607,Beoordeling.608,Beoordeling.609,Beoordeling.610,Beoordeling.all,gpt-4o_AD_PROBL_I1,gpt-4o_AD_PROBL_I2,gpt-4o_AD_PROBL_I3,gpt-4o_AD_CREAT_I1,...,gpt-4o_AD_OORDE_I2,gpt-4o_AD_OORDE_I3,gpt-4o_AD_ORGANS_I1,gpt-4o_AD_ORGANS_I2,gpt-4o_AD_ORGANS_I3,gpt-4o_AD_PROBL,gpt-4o_AD_CREAT,gpt-4o_AD_OORDE,gpt-4o_AD_ORGANS,gpt-4o_AD
0,1460133,"Oh ja, Carina ik kan me voorstellen dat je voo...","Jeetje ik, ik kan me voorstellen dat je dat la...","Ja, dus als ik het zo hoor, dan is er dus eige...","Ik denk dat het goed is als we, zoals ik net a...","Oh ja, Carina ik kan me voorstellen dat je voo...",2.0,2.0,2.0,2.0,...,1.0,2.0,2.0,2.0,2.0,2.0,1.666667,1.666667,2.000000,1.833333
16,1595547,"Dacht, Surina ja, ik begrijp je heel erg goed ...","Ja, ik begrijp je heel erg goed dat het best w...","Ja, ik zie ook echt wel dat het wat met je doe...",Maar wat ik zou aanraden is in ieder geval alv...,"Dacht, Surina ja, ik begrijp je heel erg goed ...",2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,3.0,2.0,2.0,2.000000,2.000000,2.333333,2.083333


In [ ]:
# Optional: Save results
# advise.to_csv('annotated_advise.csv', index=False)

## Codebook

The codebook below explains the column names of the datasets.  
The original column names (e.g., `Score.566`) have been re-named such that the new names indicate:  

**(a)** the job competency being evaluated (i.e., `AD` = Advisement, `CM` = Change management, `TM` = Team management, `TW` = Team work),  

**(b)** the sub-dimension being evaluated (e.g., `PROBM` = Problem analysis),  

**(c)** the number of the item of the subdimension (e.g., `I1` = item 1).  

For instance, in the column name `AD_PROBL_I1`, `AD` refers to the competency of `Advisement`, `PROBL` refers to the sub-competence of `Problem analyse`, and `I1` refers to the first item (item 1) of that sub-compentence.


**Advisement Codebook**:
| Code_name | Original_column_name | Label |
| -------- |-------------|-------|
|Beoordeling.607 |Beoordeling.607 | Transcript scene2 |
|Beoordeling.608 |Beoordeling.608 | Transcript scene3 |
|Beoordeling.609 |Beoordeling.609 | Transcript scene4 |
|Beoordeling.610 |Beoordeling.610 | Transcript scene5 |
|Beoordeling.all |Beoordeling.all | Transcripts scene2 to scene5 |
| AD_PROBL_I1 | Score.566   | Advisor's assessment: PROBL-I1_Maakt onderscheid tussen hoofd- en bijzaken in de situatie |
| AD_PROBL_I2 | Score.567   | Advisor's assessment: PROBL-I2_Legt verbanden tussen verschillende aspecten van de vraagstelling |
| AD_PROBL_I3 | Score.568   | Advisor's assessment: PROBL-I3_Benoemt de kern van de informatie |
| AD_CREAT_I1 | Score.569   | Advisor's assessment: CREAT-I1_Komt met originele ideeën |
| AD_CREAT_I2 | Score.570   | Advisor's assessment: CREAT-I2_Benadert zaken vanuit meerdere invalshoeken bij het bedenken van oplossingen/ideeën |
| AD_CREAT_I3 | Score.571   | Advisor's assessment: CREAT-I3_Combineert bestaande zaken tot iets nieuws |
| AD_OORDE_I1 | Score.1544 | Advisor's assessment: OORDE-I1_Maakt in zijn/haar/hen beeldvorming een onderscheid tussen relevante en niet relevante aspecten van een kwestie |
| AD_OORDE_I2 | Score.1545 | Advisor's assessment: OORDE-I2_Onderbouwt zijn/haar/hen redenering met feiten |
| AD_OORDE_I3 | Score.1546 | Advisor's assessment: OORDE-I3_Benoemt van eigen standpunt zowel voor- als nadelen |
| AD_ORGANS_I1 | Score.1547 | Advisor's assessment: ORGANS-I1_Houdt in het gesprek rekening met gevoeligheden van anderen in de organisatie |
| AD_ORGANS_I2 | Score.1548 | Advisor's assessment: ORGANS-I2_Toetst vooraf hoe acties/besluiten binnen de organisatie ontvangen zullen worden |
| AD_ORGANS_I3 | Score.1549 | Advisor's assessment: ORGANS-I3_Onderzoekt de gevolgen voor andere personen/organisatie-onderdelen alvorens te besluiten |


**Change management Codebook**:
| Code_name | Original_column_name | Label |
| -------- |-------------|-------|
| CM_LEIDIN_I2 | Score.615  | Oordeel adviseur: LEIDIN-I2_Geeft aandacht aan goede prestaties van de medewerker, uit waardering |
| CM_LEIDIN_I1 | Score.616  | Oordeel adviseur: LEIDIN-I1_Schept helderheid over de van de medewerker verwachte prestaties |
| CM_LEIDIN_I3 | Score.617  | Oordeel adviseur: LEIDIN-I3_Past stijl van leidinggeven aan de situatie en persoon aan |
| CM_COACH_I1 | Score.618  | Oordeel adviseur: COACH-I1_Helpt de ander inzicht te krijgen in diens sterke en zwakke punten |
| CM_COACH_I2 | Score.619  | Oordeel adviseur: COACH-I2_Stimuleert de ander om zich te ontwikkelen en nieuwe uitdagingen aan te gaan |
| CM_COACH_I3 | Score.620  | Oordeel adviseur: COACH-I3_Geeft ruimte om op een eigen manier te werken aan ontwikkeling |
| CM_VISIEU_I1 | Score.627  | Oordeel adviseur: VISIEU-I1_Benoemt de (lange termijn) doelen voor de organisatie |
| CM_VISIEU_I2 | Score.628  | Oordeel adviseur: VISIEU-I2_Geeft inzicht heeft in wat het beleid betekent voor de werkzaamheden |
| CM_VISIEU_I3 | Score.629  | Oordeel adviseur: VISIEU-I3_Legt meer de nadruk op mogelijkheden in de toekomst dan op beperkingen 
| CM_RESUL_I1 | Score.621  | Oordeel adviseur: RESUL-I1_Komt met voorstellen over hoe doelen sneller of beter kunnen worden bereikt |
| CM_RESUL_I2 | Score.622  | Oordeel adviseur: RESUL-I2_Maakt duidelijke afspraken met de medewerker over de te bereiken resultaten |
| CM_RESUL_I3 | Score.623  | Oordeel adviseur: RESUL-I3_Grijpt tijdig in wanneer resultaten niet bereikt dreigen te worden; stuurt bij |

**Team management Codebook**:
| Code_name | Original_column_name | Label |
| -------- |-------------|-------|
| TM_LEIDIN_I2 | Score.615  | Oordeel adviseur: LEIDIN-I2_Geeft aandacht aan goede prestaties van de medewerker, uit waardering |
| TM_LEIDIN_I1 | Score.616  | Oordeel adviseur: LEIDIN-I1_Schept helderheid over de van de medewerker verwachte prestaties |
| TM_LEIDIN_I3 | Score.617  | Oordeel adviseur: LEIDIN-I3_Past stijl van leidinggeven aan de situatie en persoon aan |
| TM_COACH_I1 | Score.618  | Oordeel adviseur: COACH-I1_Helpt de ander inzicht te krijgen in diens sterke en zwakke punten |
| TM_COACH_I2 | Score.619  | Oordeel adviseur: COACH-I2_Stimuleert de ander om zich te ontwikkelen en nieuwe uitdagingen aan te gaan |
| TM_COACH_I3 | Score.620  | Oordeel adviseur: COACH-I3_Geeft ruimte om op een eigen manier te werken aan ontwikkeling |
| TM_OVERT_I1 | Score.552  | Oordeel adviseur: OVERT-I1_Gebruikt argumenten die aansluiten bij de belangen of voorkeuren van de medewerker |
| TM_OVERT_I2 | Score.553  | Oordeel adviseur: OVERT-I2_Maakt duidelijk wat de voordelen van zijn/haar/diens voorstel zijn voor de medewerker |
| TM_OVERT_I3 | Score.554  | Oordeel adviseur: OVERT-I3_Maakt weerstand van de medewerker bespreekbaar |
| TM_RESUL_I1 | Score.621  | Oordeel adviseur: RESUL-I1_Komt met voorstellen over hoe doelen sneller of beter kunnen worden bereikt |
| TM_RESUL_I2 | Score.622  | Oordeel adviseur: RESUL-I2_Maakt duidelijke afspraken met de medewerker over de te bereiken resultaten |
| TM_RESUL_I3 | Score.623  | Oordeel adviseur: RESUL-I3_Grijpt tijdig in wanneer resultaten niet bereikt dreigen te worden; stuurt bij |


**Team work Codebook**:
| Code_name | Original_column_name | Label |
| -------- |-------------|-------|
| TW_SAMEN_I1 | Score.549  | Oordeel adviseur: SAMEN-I1_Biedt de collega hulp aan |
| TW_SAMEN_I2 | Score.550  | Oordeel adviseur: SAMEN-I2_Stelt gezamenlijk belang boven het eigen belang |
| TW_SAMEN_I3 | Score.551  | Oordeel adviseur: SAMEN-I3_Onderneemt acties die de samenwerking bevorderen |
| TW_OVERT_I1 | Score.552  | Oordeel adviseur: OVERT-I1_Maakt duidelijk wat de voordelen zijn van zijn/haar/diens voorstel |
| TW_OVERT_I2 | Score.553  | Oordeel adviseur: OVERT-I2_Gebruikt argumenten die aansluiten bij de belangen van de ander |
| TW_OVERT_I3 | Score.554  | Oordeel adviseur: OVERT-I3_Maakt weerstand van de ander op gepaste manier bespreekbaar |
| TW_MONDEC_I1 | Score.555  | Oordeel adviseur: MONDEC-I1_Brengt de boodschap op een voor de ander begrijpelijke manier over |
| TW_MONDEC_I2 | Score.556  | Oordeel adviseur: MONDEC-I2_Verduidelijkt de boodschap met concrete voorbeelden |
| TW_MONDEC_I3 | Score.557  | Oordeel adviseur: MONDEC-I3_Onderscheidt in zijn/haar/diens boodschap hoofd- en bijzaken op logische manier |
| TW_ORGANE_I1 | Score.558  | Oordeel adviseur: ORGANE-I1_Stelt prioriteiten voor het eigen werk; belangrijke zaken eerst |
| TW_ORGANE_I2 | Score.559  | Oordeel adviseur: ORGANE-I2_Houdt zich in het eigen werk aan met anderen gemaakte afspraken |
| TW_ORGANE_I3 | Score.560  | Oordeel adviseur: ORGANE-I3_Laat tijdig aan de collega weten wanneer het eigen werk geen nieuwe bezigheden toelaat |
